# 새 데이터 가져오기

# eda
## 사명 변경된 적 있는지
```
final_df = df[df.groupby('company_sn')['company_name'].transform('nunique') > 1].sort_values(by='job_idx')[['company_sn', 'company_name']].drop_duplicates()
final_df['name_count'] = final_df.groupby('company_sn').cumcount() + 1
final_df.pivot_table(columns='name_count', index='company_sn', values='company_name', aggfunc='first')
```

## 목록에 표시된 해시태그
- 하지만 각 공고마다 가지고 있는 태그 전체 내용이 아니다.
```
df2 = df[['job_idx','job_sectors']]
df2.explode('job_sectors').groupby('job_sectors').size().sort_values(ascending=False).to_csv('job_sectors.csv')
df.explode('job_sectors').groupby('job_sectors').get_group('크롤링')

## jobsector 데이터가 조회할때마다 달라지는지 확인 필요
- 다섯개 이상 선택해도 리스트에 표시되지 않음
- 지정된 우선순위 순서로 표시되는지 아니면 조회할때마다 무작위로 5개 선택하는지 확인 필요

# 구글 시트에서 이전 시트 가져오기

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

gc = gspread.service_account()

sh=gc.open_by_key('1vkZ7FF1bgACwXKxkmSNemmSNiti4Bd2M177w-8H80ww').sheet1




# modified          datetime64[ns]
# resistered        datetime64[ns]
# update            datetime64[ns]
# df_old의 세가지 열 데이터타입 변경


df_old = get_as_dataframe(sh, evaluate_formulas=False)
df_old.dropna(subset=['job_idx'], inplace=True)
df_old['job_idx'] = df_old['job_idx'].astype(int)
df_old.set_index('job_idx', inplace=True)
df_old['is_headhunting'] = df_old['is_headhunting'].astype(bool)
df_old["modified"] = pd.to_datetime(df_old["modified"])
df_old["resistered"] = pd.to_datetime(df_old["resistered"])
df_old["update"] = pd.to_datetime(df_old["update"])
df_old

# 새시트, 이전시트 병합
- jobsector 열 리스트 확장
- 제목 변경되는 경우 특이 케이스로 보고 따로 표시
- 수정일 : 새 시트 우선
- 작성일 : 이전 시트 우선, 원래 없다가 새로 생겨나는 경우 없을듯
- update 열 : 마지막으로 조회한 시간, 새 시트 우선
- 어떻게 합치는게 세련된 방법인지 모르겠다

- old에서 안쓰는 열은 드롭? 안돼 그러면 아우터가 아니니까

- 작성일 관측 불가면 어떻게 할지?

# Merge the dataframes
merged_df = pd.merge(df_old, df_new, how='outer', on=['job_idx'])

# Apply conditions to overlapping parts
# modified new 우선, resistered old 우선, update new 우선

merged_df["resistered"] = merged_df["resistered_x"].fillna(merged_df["resistered_y"])
merged_df["modified"] = merged_df["modified_x"].fillna(merged_df["modified_y"])
merged_df["update"] = merged_df["update_y"].fillna(merged_df["update_x"])

# Drop the columns
merged_df = merged_df.drop(columns=["resistered_x", "resistered_y", "modified_x", "modified_y", "update_x", "update_y"])
merged_df

# 새 시트로 덮어쓰기

# Clear the contents of the sheet
sh.clear()

# Set the new dataframe in the sheet
set_with_dataframe(sh, combined_df, include_index=True, allow_formulas=False)


# 키워드 처리
- job_sector 열에 키워드 들어있다.
- 키워드는 리스트 타입으로
- 각 공고마다 5개 이상의 키워드 가질 수 있다.
- 공고 목록에 표시되는 건 5개의 키워드 뿐
- 키워드로 목록을 새로 조회할 수 있다.
- 키워드로 목록을 조회할 때, 해당 목록의 모든 공고가 그 키워드를 가지고 있다고 등록해줘야함
- 자동차 라는 키워드로 목록 조회
- 그 목록을 1차 처리
- 키워드 리스트를 별도 테이블로 분리하고, 모든 공고에 검색 키워드 추가


전산총무 : 415
자동차 : 2218


In [1]:
import pandas as pd
import pickle

In [2]:
# Specify the file path of the pickle file
file_path = 'df_key.pickle'

# Load the pickle file
with open(file_path, 'rb') as file:
    df_key_old = pickle.load(file)


In [3]:
# Specify the file path of the pickle file
file_path = 'df_combined.pickle'

# Load the pickle file into df_old
with open(file_path, 'rb') as file:
    df_old = pickle.load(file)


In [4]:
df_old

,career_info,company_name,company_sn,deadline,education_info,is_headhunting,job_link,job_sectors,job_title,modified,resistered,support_date,update,work_place
job_idx,,,,,,,,,,,,,,
46367962,신입 · 경력 · 정규직 외,(주)한양기술공업,WHlqeXZqTWRZVWFlL2ZkQk4zVFA4dz09,23시간 전 수정,고졸↑,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[총무, 문서작성, 비품관리, 서류관리, 시설관리, 전산총무]",㈜한양기술공업 상반기 각 부문 수시채용 공고,2024-04-01 18:25:02.691872,2024-04-01 18:25:02.691872,~05.21(화),2024-04-02 17:25:02.691872,경기 화성시 외
47327498,신입 · 정규직,(주)아모텍,ckdXSlNOd3BXV2FJcjRmeHdqNXpZZz09,85일 전 등록,"대학(2,3년)↑",False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[기술지원, 데이터엔지니어, 백엔드/서버개발, 유지보수, 정보보안, 전산총무]",(주)아모텍 각 부문별 신입사원 모집,NaT,2024-01-08 17:25:02.691872,~04.30(화),2024-04-02 17:25:02.691872,서울 서초구 외
47328813,신입 · 정규직,(주)아모텍,ckdXSlNOd3BXV2FJcjRmeHdqNXpZZz09,23시간 전 수정,"대학(2,3년)↑",False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[기술지원, 데이터엔지니어, 백엔드/서버개발, 유지보수, 정보보안, 전산총무]",(주)아모텍 각 부문별 경력사원 모집,2024-04-01 18:25:02.676247,2024-04-01 18:25:02.676247,~04.30(화),2024-04-02 17:25:02.676247,서울 서초구 외
47447368,신입 · 경력 · 정규직 외,이에이트(주),OElPTVpmV0pLL0ZtVkl3RU5Cc3lkZz09,5시간 전 수정,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[백엔드/서버개발, 보안컨설팅, 앱개발, 정보보안, 프론트엔드, 전산총무]",이에이트 신입 및 경력 채용,2024-04-02 12:25:02.676247,2024-04-02 12:25:02.676247,~05.31(금),2024-04-02 17:25:02.676247,서울 송파구
47448498,신입 · 경력 · 정규직,교육지대(주),RlNrUnZiMVRBQ2ROVlN3Nlg4dVZBUT09,6시간 전 수정,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[유지보수, 총무, 문서작성, 비품관리, 사내행사, 전산총무]",[교육지대] 경영지원실 총무 채용,2024-04-02 11:25:02.676247,2024-04-02 11:25:02.676247,D-6,2024-04-02 17:25:02.676247,서울 구로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47915183,신입 · 경력 · 정규직,㈜타임텍,NUIrbldqalF0ZkJYSTM1OWZsTE5idz09,8분 전 등록,대학교(4년)↑,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,[C++],(경력/신입) 인공지능(AI) 프로그래머 정규직 채용,NaT,2024-04-02 16:42:37.419687,~05.02(목),2024-04-02 16:50:37.419687,부산 해운대구
47915187,3 ~ 6년 · 정규직,(주)엠투아이소프트,WngwU2QyTTZCaEp3MGpUZU9rcC84QT09,8분 전 등록,대학교(4년)↑,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[웹개발, ASP, PHP]",WEB 프로그래머 모집,NaT,2024-04-02 16:42:37.419687,~05.02(목),2024-04-02 16:50:37.419687,대구 수성구
47915191,경력 2년↑ · 정규직,(주)굿인벤트,YU8vUFdqVnFFTlQwRFFrQlhlL01CZz09,7분 전 등록,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[백엔드/서버개발, 웹개발, 프론트엔드, ASP, Java]",[정규직/여의도] 기획개발팀 웹개발자 경력 모집,NaT,2024-04-02 16:43:37.419687,~05.02(목),2024-04-02 16:50:37.419687,서울 영등포구
